1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
sns.set_context("paper", font_scale=2)

In [2]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [3]:
def calc_logloss(y, y_pred):
    y_pred_clipped = np.clip(y_pred, 1e-10, 0.9999999999)
    err = np.mean(- y * np.log(y_pred_clipped) - (1.0 - y) * np.log(1.0 - y_pred_clipped))
    return err

In [4]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика
X

array([[1.00e+00, 1.00e+00, 5.00e+02, 1.00e+00],
       [1.00e+00, 1.00e+00, 7.00e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 7.50e+02, 2.00e+00],
       [1.00e+00, 1.00e+00, 6.00e+02, 1.00e+00],
       [1.00e+00, 3.00e+00, 1.45e+03, 2.00e+00],
       [1.00e+00, 0.00e+00, 8.00e+02, 1.00e+00],
       [1.00e+00, 5.00e+00, 1.50e+03, 3.00e+00],
       [1.00e+00, 1.00e+01, 2.00e+03, 3.00e+00],
       [1.00e+00, 1.00e+00, 4.50e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 1.00e+03, 2.00e+00]])

In [5]:
def standard_scaler(x):
    res = (x - x.mean()) / x.std()
    return res

X_st = X.copy()
X_st[:, 2] = standard_scaler(X[:, 2])
X_st[:, 2]

array([-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
       -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735])

In [6]:
def eval_LR_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    err = 0
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * np.dot(X.T, (y_pred - y)))
        '''if i % 10 == 0:
            print(i, w, err, alpha)'''
    return w, err

2.Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss был минимальным (оптимально здесь использовать критерий остановки обучения).

In [7]:
diff = 1e-6
err = 0
err_new = 0
res = []
for i in range(1000, 3001, 100):
    for a in np.linspace(0.01, 1, num=100):
        w, err_new = eval_LR_model(X_st, y, i, a)
        if (err_new - err) < 1e-6:
            res = [i, w, err_new, a]            
        err = err_new
print(res)

[3000, array([-20.15246511,  -2.20833679,  -5.07286862,  16.68110185]), 0.18207622786745659, 1.0]


In [8]:
w, err = eval_LR_model(X_st, y, iterations=3000, alpha=1)

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, на выходе - массив y_pred_proba).

In [9]:
def calc_pred_proba(X ,w):
    y = X @ w
    proba = sigmoid(y)
    return proba

In [10]:
proba = calc_pred_proba(X_st, w)
proba

array([0.32951703, 0.05717739, 0.99998558, 0.14722359, 0.83415954,
       0.16238921, 0.99999842, 0.05137714, 0.45331565, 0.99980288])

4.Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, на выходе - массив y_pred).

In [11]:
def calc_pred(X, w):
    y_pred_proba = calc_pred_proba(X, w)
    for i in range(len(y_pred_proba)):
        if y_pred_proba[i] < 0.5:
            y_pred_proba[i] = 0
        elif y_pred_proba[i] >= 0.5:
            y_pred_proba[i] = 1
    return y_pred_proba

In [12]:
y_pred = calc_pred(X_st, w)
y_pred

array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.])

In [13]:
y

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

5.Посчитайте accuracy, матрицу ошибок, precision и recall, а также F1-score.

In [14]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(len(y)):
    if y_pred[i] == y[i] == 1:
        TP+=1
    elif y_pred[i] == 1 and y[i] == 0:
        FP+=1
    elif y_pred[i] == y[i] == 0:
        TN+=1
    elif y_pred[i] == 0 and y[i] ==1:
        FN+=1
print('Confusion matrix:\n',np.array([[TP, FP],[FN, TN]]))

Confusion matrix:
 [[4 0]
 [1 5]]


In [15]:
accuracy = (TP+TN)/len(y)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9


In [16]:
precision = TP/(TP+FP)
print(f'Precision: {precision}')

Precision: 1.0


In [17]:
recall = TP/(TP+FN)
print(f'Recall: {recall}')

Recall: 0.8


In [18]:
f1_score = (2*precision*recall)/(precision+recall)
print(f'F1 score: {f1_score}')

F1 score: 0.888888888888889


6.Могла ли модель переобучиться? Почему?

Как и модель линейной регрессии, логистическая подвержена переобучению из-за способности подстраивать коэффициенты под обучающую выборку(минимизируя log-loss) вследствие чего снижается обощающая способность алгоритма.